In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
! ls /content/drive/MyDrive/Colab_Notebooks/dataset_nndl

camelyonpatch_level_2_split_test_y.h5.gz.gzip	camelyonpatch_level_2_split_valid_y.h5.gz.gzip
camelyonpatch_level_2_split_train_y.h5.gz.gzip


In [ ]:
!ls
!pwd
#! cp /content/drive/MyDrive/Colab_Notebooks/dataset_nndl/camelyon* ./


camelyonpatch_level_2_split_test_y.h5	camelyonpatch_level_2_split_valid_y.h5	sample_data
camelyonpatch_level_2_split_train_y.h5	drive
/content


In [ ]:
! mv camelyonpatch_level_2_split_test_y.h5.gz.gzip  camelyonpatch_level_2_split_test_y.h5.gz
! mv camelyonpatch_level_2_split_valid_y.h5.gz.gzip  camelyonpatch_level_2_split_valid_y.h5.gz
! mv camelyonpatch_level_2_split_train_y.h5.gz.gzip  camelyonpatch_level_2_split_train_y.h5.gz
! gzip -df camelyonpatch_level_2_split_test_x.h5.gz
! gzip -df camelyonpatch_level_2_split_test_y.h5.gz
! gzip -df camelyonpatch_level_2_split_valid_x.h5.gz
! gzip -df camelyonpatch_level_2_split_valid_y.h5.gz
! gzip -df camelyonpatch_level_2_split_train_x.h5.gz
! gzip -df camelyonpatch_level_2_split_train_y.h5.gz

gzip: camelyonpatch_level_2_split_test_x.h5.gz: No such file or directory
gzip: camelyonpatch_level_2_split_valid_x.h5.gz: No such file or directory
gzip: camelyonpatch_level_2_split_train_x.h5.gz: No such file or directory


In [ ]:
from torchvision.transforms import Compose, ToTensor, RandomAffine, RandomHorizontalFlip, RandomVerticalFlip, ColorJitter
import torch
import os
import pandas as pd
import h5py
import random
import numpy as np

seed = 0
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

In [ ]:
transforms = Compose([
    ToTensor(), #this converts numpy or Pil image to torch tensor and normalizes it in 0, 1
    RandomAffine((0.05, 0.05)),
    RandomHorizontalFlip(),
    RandomVerticalFlip()
])

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class ImageDataset(Dataset):
  def __init__(self, dataset_folder, dataset_type, transform=None):
    self.x = h5py.File(os.path.join(dataset_folder, f'camelyonpatch_level_2_split_{dataset_type}_x.h5'), 'r')['x']
    self.y = h5py.File(os.path.join(dataset_folder, f'camelyonpatch_level_2_split_{dataset_type}_y.h5'), 'r')['y']
    self.transform = transform

  def __len__(self):
    return min(3200, len(self.x))

  def __getitem__(self, idx):
    x = self.x[idx]
    y = slef.y[idx]
    if self.transform:
      x = self.transform(x)
    return x,y

In [ ]:
train_dataset = ImageDataset('./','train', transforms)
valid_dataset = ImageDataset('./', 'valid', ToTensor())
test_dataset = ImageDataset('./', 'test', ToTensor())


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = './camelyonpatch_level_2_split_train_x.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(4, 4, figsize=(6,6))
for i in range(16):
    axs[i//4][i%4].imshow(train_dataset[i][0].permute(1, 2, 0).numpy())
    axs[i//4][i%4].set_xticks([])
    axs[i//4][i%4].set_yticks([])
    axs[i//4][i%4].set_title(f"class: {train_dataset[i][1]}")
plt.tight_layout()

In [ ]:
batch_size=64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=os.cpu_count())
valid_dataset = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=os.cpu_count())
test_dataset = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=os.cpu.count())


In [ ]:
from torch.nn import Module, Sequential, Conv2d, BatchNorm2d, ReLU


In [ ]:
class MainPath(Module):
  def __init__(self, in_channels, filters, kernel_size, stride=1):
    super().__init__()
    F1, F2, F3 = filters
    self.main_path = Sequential(
        Conv2d(in_channels, F1, kernel_size=1, stride=stride),
        BatchNorm2d(F1),
        ReLU(),
        Conv2d(F1, F2, kernel_size=kernel_size, padding=kernel_size//2),
        BatchNorm2d(F2),
        ReLU(),
        Conv2d(F2, F3, kernel_size=1),
        BatchNorm3d(F3)
        )
        self.apply(self._init_weights)

    def _init_weights(self, module):
      if isinstance(module, torch.nn.Linear):
        torch.nn.init.xavier_uniform_(module.weight)
        if module.bias is not None:
          module.bias.data.zero_()
      if isinstance(module, torch.nn.Conv2d):
        torch.nn.init_xavier_uniform_(module.weight)
        if module.bias is not None:
          module.bias.data.zero_()

    def forward(self, x):
      y = self.main_path(x)
      return y


class IdentityBlock(MainPath):
  def __init__(self, in_channels, filters, kernel_size):
    super().__init__(in_channels, filters, kernel_size)
    self.relu = ReLU()

  def forward(self, x):
    y = self.relu(self.main_path(x)+x)
    return y


In [ ]:
id_block = IdentityBlock(3, [100, 100, 3], 3)
print(id_block(torch.zeros(1, 3, 10, 10)))
print('Expected block')

In [ ]:
class ConvolutionalBlock(MainPath):
  def __init__(self, in_channels, filters, kernel_size):
    super().__init__(in_channels, filters, kernel_size, stride=2)
    self.relu = ReLU()
    self.shortcut_path = Sequential(Conv2d(in_channels, filters[2], kernel_size=1, stride=2),
                                    BatchNorm2d(filters[2]))
    self.apply(self._init_weights)

    def _init_weights(self, module):
      if isinstance(module, torch.nn.Linear):
        torch.nn.init_xavier_uniform_(module.weight)
        if module.bias is not None:
          module.bias.data.zero_()
      if isinstance(module, torch.nn.Conv2d):
        torch.nn.init_xavier_uniform_(module.weight)
        if module.bias is not None:
          module.bias.data.zero_()
    def forward(self, x):
      y = self.relu(self.main_path(x) + self.shortcut_path(x))
      return y


In [ ]:
id_block = ConvolutionalBlock(3, [100, 100, 3], 3)
print(id_block(torch.zeros(1, 3, 10, 10)).shape)
print("expected shape: (1, 3, 5, 5)")

In [ ]:
from torch.nn import MaxPool2d, AvgPool2d, Linear, Dropout


In [ ]:
class ResNet50(Module):

  def __init__(self):
    super().__init__()
    self.network = Sequential(
        Conv2d(3, 64, kernel_size=7, stride=2),
        BatchNorm2d(64),
        MaxPool2d(kernel_size=3, stride=2),
        ConvolutionalBlock(64, [64, 64, 256], kernel_size=3),
        Dropout(0.2),
        IdentityBlock(256, [64, 64, 256], kernel_size=3),
        IdentityBlock(256, [64, 64, 256], kernel_size=3),
        ConvolutionalBlock(256, [128, 128, 512], kernel_size=3),
        Dropout(0.2),
        IdentityBlock(512, [128, 128, 512], kernel_size=3),
        IdentityBlock(512, [128, 128, 512], kernel_size=3),
        IdentityBlock(512, [128,128, 512], kernel_size=3),
        ConvolutionalBlock(512, [256, 256, 1024], kernel_size=3),
        Dropout(0.2),
        IdentityBlock(1024, [256, 256, 1024], kernel_size=3),
        IdentityBlock(1024, [256, 256, 1024], kernel_size=3),
        IdentityBlock(1024, [256, 256, 1024], kernel_size=3),
        IdentityBlock(1024, [256, 256, 1024], kernel_size=3),
        IdentityBlock(1024, [256, 256, 1024], kernel_size=3),
        ConvolutionalBlock(1024, [512, 512, 2048], kernel_size=3),
        Dropout(0.2),
        IdentityBlock(2048, [512, 512, 2048], kernel_size=3),
        IdentityBlock(2048, [512, 512, 2048], kernel_size=3),
        AvgPool2d(kernel_size=2, stride=2),
        )

      self.classification_layer = Linear(2048,1)
      self.apply(self._init_weights)

    def forward(self, x):
      y = self.network(x).reshape((x.shape[0], -1))
      y = self.classification_layer(y)
      return y

    def _init_weights(self, module):
      if isinstance(module, torch.nn.Linear):
        torch.nn.init_xavier_uniform_(module.weight)
        if module.bias is not None:
          module.bias.data.zero_()
      if isinstace(module, torch.nn.Conv2d):
        torch.nn.init_xavier_uniform_(module.weight)
        if module.bias is not None:
          module.bias.data.zero_()



In [ ]:
from torch.optim import SGD, Adam
from torch.nn import BCEWithLogitsLoss
from tqdm import tqdm

In [ ]:
model = ResNet50()
opt = SGD(model.parameters(), lr=1e-2, weight_decay=0)
loss_fn = BCEWithLogitsLoss()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
epochs=10
best_val = np.inf
for epoch in range(epochs):
  model.train()
  print(f'Epoch: {epoch+1}')
  iterator = tqdm(train_dataloader)
  for batch_x, batch_y in iterator:
    batch_x = batch_x.to(device)
    batch_y = batch_y.to(device)
    y_pred = model(batch_x)
    loss = loss_fn(y_pred, batch_y)
    opt.zero_grad()
    loss.backward()
    opt.step()
    iterator.set_description(f'Train Loss: {loss.detach().cpu().numpy()}')

  model.eval()
  with torch.no_grad():
    predictions=[]
    true=[]
    for batch_x, batch_y in tqdm(valid_dataloader):
      batch_x = batch_x.to(device)
      batch_y = batch_y.to(device)

      y_pred = model(batch_x)
      predictions.append(y_pred)
      true.append(batch_y)
    predictions = torch.cat(predictions, axis=0)
    true = torch.cat(true, axis=0)
    val_loss = loss_fn(predictions, true)
    val_acc = (torch.sigmoid(predictions).round() == true).float().mean()
    print(f'Val loss {val_loss}, Val acc {val_acc}')

  if val_loss < best_val:
    print("Saved Model")
    torch.save(model.state_dict(), 'model.pt')
    best_val = val_loss









In [ ]:
from sklearn.metrics import accuracy_score, precosion_recall_fscore_support, roc_curve, roc_auc_score

def evaluate_network(dataloader, model, data_split):
  model.eval()
  with torch.no_grad():
    predictions=[]
    true=[]
    for batch_x, batch_y in tqdm(dataloader):
      batch_x = batch_x.to(device)
      batch_y = batch_y.to(device)
      y_pred = model(batch_x)
      predictions.append(y_pred)
      true.append(batch_y)
    predictions = torch.cat(predictions, axis=0)
    true = torch.cat(true, axis=0)
    loss = loss_fn(predictions, true).detach().cpu().numpy()
    predictions = torch.sigmoid(predictions).detach().cpu().numpy()
    true = true.detach().cpu().numpy()

    fpr, tpr, thresholds = roc_curve(true, predictions)
    auc = roc_auc_score(true, predictions)
    predictions = predictions.round()
    precision, recall, fscore, _ = precision_recall_fscore_support(true, predictions, average='binary')
    accuracy = accuracy_score(true, predictions)

    print(f"{data_split} loss: {loss}, accuracy: {accuracy}, precision: {precision}, recall: {recall}, f1: {fscore}, roc_auc: {auc}")

        plt.figure()
        plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % auc)
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'{data_split} receiver operating characteristic (ROC)')
        plt.legend(loc="lower right")






In [ ]:
model.load_state_dict(torch.load("model.pt"))
evaluate_network(train_dataloader, model, "Training Dataset")
evaluate_network(valid_dataloader, model, "Validation Dataset")
evaluate_network(test_dataloader, model, "Test Dataset")

In [ ]:
#Inception Network


In [ ]:
# 2d Convolution with Batch Normalization
class Conv2d_bn(Module):
  #kernel_size, padding and stride can be different per each one of the two dimensions to convolute. So each feature is weighted differently.
  def __init__(self, in_filters, out_filters, kernel_size, strides, padding):
    super().__init__():
    if isinstance(kernel_size, tuple):
      padding_val = (kernel_size//2 for k in kernel_size) if padding == 'same' else (0,0)
    else:
      padding_val = kernel_size//2 if padding == 'same' else 0
      self.conv = Conv2d(in_filters, out_filters, kernel_size, strides, padding_val)
      self.bn = BatchNorm2d(out_filters)
      self.relu = ReLU()
      self.apply(self._init_weights)
  def _init_weights(self, module):
    if isinstance(module, torch.nn.Linear):
      torch.nn.init_xavier_uniform_(module.weight)
      if module.bias is not None:
        module.bias.data.zero_()
    if isinstance(module, torch.nn.Conv2d):
      torch.nn.init_xavier_uniform_(module.weight)
      if module.bias is not None:
        module.bias.data.zero_()
  def _forward(self, x):
    return self.relu(self.bn(self.conv(x)))






In [ ]:
#Stem Block
class StemBlock(Module):
  def __init__(self):
    super().__init__()
    self.first_block = sequential(
        Conv2d_bn(in_filters=3, out_filters=32, kernel_size=3, strides=2, padding='valid'),
        Conv2d_bn(in_filters=32, out_filters=32, kernel_size=3, strides=1, padding='valid'),
        Conv2d_bn(in_filters=32, out_filters=64, kernel_size=3, strides=1, padding='same'),
        )
    self.first_left = MaxPool2d(kernel_size=3, stride=2, padding=0)
    self.first_right = Conv2d_bn(in_filters=64, out_filters=96, kernel_size=3, stride=2, padding='valid')

    self.second_left = Sequential(
        Conv2d_bn(in_filters=160, out_filters=64, kernel_size=1, strides=1, padding='same'),
        Conv2d_bn(in_filters=64, out_filters=96, kernel_size=3, strides=1, padding='valid'),
    )
    self.second_right = Sequential(
        Conv2d_bn(in_filters=160, out_filters=64, kernel_size=1, strides=1, padding='same'),
        Conv2d_bn(in_filters=64, out_filters=64, kernel_size=(7,1), strides=1, padding='same'),
        Conv2d_bn(in_filters=64, out_filters=64, kernel_size=(1,7), strides=1, padding= 'same'),
        Conv2d_bn(in_filters=64, out_filters=96, kernel_size=3, strides=1, padding='valid'),
    )

    self.third_left = Conv2d_bn(in_filters=192, out_filters=192, kernel_size=3, strides=2, padding='valid')
    self.third_right = MaxPool2d(kernel_size=3, stride=2, padding=0)

    self.apply(self._init_weights)

  def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv2d):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()


  def forward(self, x):
    x = self.first_block(x)
    x_l = self.first_left(x)
    x_r = self.first_right(x)
    x = torch.cat([x_l, x_r], axis=1)
    x_l = self.second_left(x)
    x_r = self.second_right(x)
    x = torch.cat([x_l, x_r], axis=1)
    x_l = self.third_left(x)
    x_r = self.third_right(x)
    x = torch.cat([x_l, x_r], axis=1)

    return x


In [ ]:
model = StemBlock()
print(model(torch.zeros((1,3,299,299))).shape)
print(f"Expected shape (1, 384, 35, 35)")

In [ ]:
# Inception A block
class A_block(Module):
  def __init__(self, in_filters):
    super().__init__()
    self.avg_block = Sequential(
        AvgPool2d(kernel_size=3, stride=1, padding=1),
        Conv2d_bn(in_filters=in_filters, out_filters=96, kernel_size=1, strides=1, padding='same'),
    )

    self.one_by_one_block = Conv2d_bn(in_filters=in_filters, out_filters=96, kernel_size=1, strides=1, padding='same'),

    self.three_by_three_block = Sequential(
        Conv2d_bn(in_filters= in_filters, out_filters=64, kernel_size=1, strides=1, padding='same'),
        Conv2d_bn(in_filters=64, out_filters=96, kernel_size=3, strides=1, padding='same'),
    )

    self.five_by_five_block = Sequential(
        Conv2d_bn(in_filters=in_filters, out_filters=64, kernel_size=1, strides=1, padding='same'),
        Conv2d_bn(in_filters=64, out_filters=96, kernel_size=3, strides=1, padding='same'),
        Conv2d_bn(in_filters=96, out_filters=96, kernel_size=3, strides=1, padding='same'),
    )

    self.apply(self._init_weights)

  def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv2d):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()

  def forward(self, x):
    x_1 = self.avg_block(x)
    x_2 = self.one_by_one_block(x)
    x_3 = self.three_by_three_block(x)
    x_4 = self.five_by_five_block(x)
    x = torch.cat([x_1, x_2, x_3, x_4], axis=1)

    return x

In [ ]:
a_block = A_block(384)
print(a_block(torch.zeros((1, 384, 35, 35))).shape)
print(f"Expected shape (1, 384, 35, 35)")

In [ ]:
#Inception_B block
class B_block(Module):
  def __init__(self, in_filters):
    super().__init__()
    self.avg_block = Sequential(
        AvgPool2d(kernel_size=3, stride=1, padding=1),
        Conv2d_bn(in_filters=in_filters, out_filters=128, kernel_size=1, strides=1, padding='same'),
    )

    self.one_by_one_block = Conv2d_bn(in_filters=in_filters, out_filters=384, kernel_size=1, strides=1, padding= 'same')

    self.seven_by_seven_block = Sequential(
        Conv2d_bn(in_filters=in_filters, out_filters=192, kernel_size=1, strides=1, padding='same'),
        Conv2d_bn(in_filters=192, out_filters=224, kernel_size=(1,7), strides=1, padding='same'),
        Conv2d_bn(in_filters=224, out_filters=256, kernel_size=(7,1), strides=1, padding='same'),
    )

    self.thirteen_by_thirteen_block = Sequential(
        Conv2d_bn(in_filters=in_filters, out_filters=192, kernel_size=1, strides=1, padding='same'),
        Conv2d_bn(in_filters=192, out_filters=192, kernel_size=(1,7), strides=1, padding='same'),
        Conv2d_bn(in_filters=192, out_filters=224, kernel_size=(7,1), strides=1, padding='same'),
        Conv2d_bn(in_filters=224, out_filters=224, kernel_size=(1,7), strides=1, padding='same'),
        Conv2d_bn(in_filters=224, out_filters=256, kernel_size=(7,1), strides=1, padding='same'),
    )

    self.apply(self._init_weights)

  def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv2d):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()


  def forward(self, x):
    x1 = self.avg_block(x)
    x2 = self.one_by_one_block(x)
    x3 = self.seven_by_seven_block(x)
    x4 = self.thirteen_by_thirteen_block(x)
    x = torch.cat([x1, x2, x3, x4], axis = 1)

    return x

In [ ]:
b_block = B_block(1024)
print(b_block(torch.zeros((1, 1024, 17, 17))).shape)
print(f"Expected shape (1, 1024, 17, 17)")

In [ ]:
#Inception C block
class C_block(Module):
  def __init__(self, in_filters):
    super().__init__()
    self.avg_block = Sequential(
        Avg_Pool2d(kernel_size=3, stride=1, padding=1),
        Conv2d_bn(in_filters=in_filters, out_filters=256, kernel_size=1, strides=1, padding='same'),
    )

    self.one_by_one_block = Conv2d_bn(in_filters=in_filters, out_filters=256, kernel_size=1, strides=1, padding='same')

    self.branch_a = Conv2d_bn(in_filters=in_filters, out_filters=384, kernel_size=1, strides=1, padding='same')

    self.branch_a_left = Conv2d_bn(in_filters=384, out_filters=256, kernel_size=(1,3), strides=1, padding='same')

    self.branch_a_right = Conv2d_bn(in_filters=384, out_filters=256, kernel_size=(3,1), strides=1, padding='same')

    self.branch_b = Sequential(
        Conv2d_bn(in_filters=in_filters, out_filters=384, kernel_size=1, strides=1, padding='same'),
        Conv2d_bn(in_filters=384, out_filters=448, kernel_size=(1,3), strides=1, padding='same'),
        Conv2d_bn(in_filters=448, out_filters=512, kernel_size=(3,1), strides=1, padding='same'),
    )

    self.branch_b_left = Conv2d_bn(in_filters=512, out_filters=256, kernel_size=(1,3), strides=1, padding='same')

    self.branch_b_right = Conv2d_bn(in_filters=512, out_filters=256, kernel_size=(3,1), strides=1, padding='same')

    self.apply(self._init_weights)

  def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv2d):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()

  def forward(self, x):
    x_1 = self.avg_block(x)
    x_2 = self.one_by_one_block(x)
    x_a = self.branch_a(x)
    x_3 = self.branch_a_left(x_a)
    x_4 = self.branch_a_right(x_a)
    x_b = self.branch_b(x)
    x_5 = self.branch_b_left(x_b)
    x_6 = self.branch_b_right(x_b)
    x = torch.cat([x_1, x_2, x_3, x_4, x_5, x_6], axis=1)

    return x

In [ ]:
c_block = C_block(1536)
print(c_block(torch.zeros(1, 1536, 8, 8)).shape)
print(f"Expected shape (1, 1536, 8, 8)")

In [ ]:
#Reduction A block
class Reduction_A(Module):
  def __init__self(self, in_filters):
    super().__init__()
    self.max_pool = MaxPool2d(kernel_size=3, stride=2, padding=0)

    self.central_block = Conv2d_bn(in_filters=in_filters, out_filters=384, kernel_size=3, strides=2, padding='valid')

    self.right_block = Sequential(
        Conv2d_bn(in_filters=in_filters, out_filters=192, kernel_size=1, stride=1, padding='same'),
        Conv2d_bn(in_filters=192, out_filters=224, kernel_size=3, strides=1, padding='same'),
        Conv2d_bn(in_filters=224, out_filters=256, kernel_size=3, strides=2, padding='valid'),
    )

    self.apply(self._init_weights)

  def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv2d):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()

  def forward(self, x):
    x_1 = self.max_pool(x)
    x_2 = self.central_block(x)
    x_3 = self.right_block(x)

    x = torch.cat([x_1, x_2, x_3], axis=1)

    return x

In [ ]:
reduction_A = Reduction_A(384)
print(reduction_A(torch.zeros(1, 384, 35, 35)).shape)
print("Expected shape (1, 1024, 17, 17)")

In [ ]:
#Reduction b block
class Reduction_B(Module):
  def __init__(self, in_filters):
    super().__init__()
    self.max_pool = MaxPool2d(kernel_size=3, stride=2, padding=0)

    self.central_block = Sequential(
        Conv2d_bn(in_filters=in_filters, out_filters=192, kernel_size=1, strides=1, padding='same'),
        Conv2d_bn(in_filters=192, out_filters=192, kernel_size=3, strides=2, padding='valid'),
    )

    self.right_block = Sequential(
        Conv2d_bn(in_filters=in_filters, out_filters=256, kernel_size=1, strides=1, padding='same'),
        Conv2d_bn(in_filters=256, out_filters=256, kernel_size=(1,7), strudes=1, padding='same'),
        Conv2d_bn(in_filters=256, out_filters=320, kernel_size=(7,1), strides=1, padding='same'),
        Conv2d_bn(in_filters=320, out_filters=320, kernel_size=3, strides=2, padding='valid'),
    )

    self.apply(self._init_weights)

  def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv2d):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()


  def forward(self, x):
        x_1 = self.max_pool(x)
        x_2 = self.central_block(x)
        x_3 = self.right_block(x)
        x = torch.cat([x_1, x_2, x_3], axis=1)
        return x

In [ ]:
from torch.nn import Dropout

In [ ]:
class InceptionV4(Module):
  def __init__(self):
    super().__init__()
    self.stem = StemBlock()
    self.inception_a = Sequential(
        A_block(384),
        A_block(384),
        A_block(384),
        A_block(384)
    )

    self.reduction_a = Reduction_A(384)

    self.inception_b = Sequential(
        B_block(1024),
        B_block(1024),
        B_block(1024),
        B_block(1024),
        B_block(1024),
        B_block(1024),
        B_block(1024)
    )

    self.reduction_b = reduction_B(1024)

    self.inception_c = Sequential(
            C_block(1536),
            C_block(1536),
            C_block(1536)
        )

    self.drop = Dropout(0.2)

    self.out = Linear(1536, 1)

    self.apply(self._init_weights)

  def forward(self, x):
    x = self.stem(x)
    x = self.inception_a(x)
    x = self.reduction_a(x)
    x = self.inception_b(x)
    x = self.reduction_b(x)
    x = self.inception_c(x)
    x = x.reshape(x.shape[0], -1, 1536).mean(axis=1)
    x = self.drop(x)
    x = self.out(x)

    return x

  def _init_weights(self, module):
    if isinstance(module, torch.nn.Linear):
        torch.nn.init.xavier_uniform_(module.weight)
        if module.bias is not None:
            module.bias.data.zero_()
    if isinstance(module, torch.nn.Conv2d):
        torch.nn.init.xavier_uniform_(module.weight)
        if module.bias is not None:
            module.bias.data.zero_()


In [ ]:
inception_v4 = InceptionV4()
print(inception_v4(torch.zeros(1, 3, 299, 299)).shape)
print("Expected shape (1, 1)")

In [ ]:
model = InceptionV4()
opt = SGD(model.parameters(), lr=0.005)
loss_fn = BCELogitsLoss()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
epochs=10

best_val=np.inf

for epoch in range(epochs):
  model.train()
  print(f'Epoch N: {epoch +1}')
  iterator = tqdm(train_dataloader)
  for x_batch, y_batch in iterator:
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)
    y_pred = model(x_batch)

    loss = loss_fn(y_pred, y_batch)
    opt.zero_grad()
    loss.backward()
    opt.step()
    iterator.set_description(f'Train_loss {loss.detach().cpu().numpy()}')

  model.eval()
  with torch.no_grad():
    predictions=[]
    true=[]
    for x_batch, y_batch in tqdm(val_dataloader):
      x_batch = x_batch.to(device)
      y_batch = y_batch.to(device)
      y_pred = model(x_batch)
      predictions.append(y_pred)
      true.append(y_batch)
    predictions = torch.cat(predictions, axis=0)
    true = torch.cat(true, axis=0)
    val_loss = loss_fn(predictions, true)

    val_acc = (torch.sigmoid(predictions).round() == true).float().mean()
    print(f'loss: {val_loss}, accuracy: {val_acc}')

    if val_loss < best_val:
      print("Saved Model")
      torch.save(model.state_dict(), 'model.pt')
      best_val = val_loss



In [ ]:
model.load_state_dict(torch.load("model.pt"))
evaluate_network(train_dataloader, model, "Training Dataset")
evaluate_network(valid_dataloader, model, "Validation Dataset")
evaluate_network(test_dataloader, model, "Test Dataset")